## First, let's  test the ping method (GET /ping)
This method will be used by Sagemaker for health check our model. It must return a code **200**.

In [ ]:
import json
from urllib import request

base_url='http://localhost:8080'

In [ ]:
resp = request.urlopen("%s/ping" % base_url)
print("Response code: %d" % resp.getcode() )

## Then we can the predictions (POST /invocations)
This method will be used by Sagemaker for the predictions. Here we're simulating the header parameter related to the CustomAttributes

In [ ]:
%%time

import pandas as pd
from io import StringIO

# Load sample data
df_test = pd.read_csv('input/data/test/test.csv').head()
payload = df_test.to_csv(index=False).encode('utf-8')

headers = {"Content-type": "text/csv"}
req = request.Request("%s/invocations" % base_url, data=payload, headers=headers)
resp = request.urlopen(req)
print("Response code: %d" % (resp.getcode()))

In [ ]:
# Print the predictions
test_df = pd.read_csv(StringIO(resp.read().decode('utf-8')))
test_df[['class_predictions','class_probability']]

## Now, you can go back to the previous Jupyter, stop it and continue executing it